In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

import plotly.tools as tls
import plotly
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import warnings
from collections import Counter

from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.stats import chisquare
from scipy.special import stdtr
from scipy.stats import chi2_contingency 



from wordcloud import WordCloud, STOPWORDS
import nltk.tokenize as word_tokenize
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
from nltk.stem.porter import *
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction import stop_words

from wordcloud import WordCloud, STOPWORDS

ModuleNotFoundError: No module named 'plotly'

In [ ]:
df_2018 = pd.read_csv("5523_Group_Project/Data/ks-projects-201801.csv")
df_2018.head()

In [ ]:
df_2018.shape
df_2018.info()
df_2018.describe()

In [ ]:
df_2018.isnull().sum()

In [ ]:
df_2018.isna().sum()

In [ ]:
df_2018.drop(['usd pledged'], axis=1, inplace=True)

In [ ]:
df_2018.dropna(inplace=True)

In [ ]:
df_2018.nunique()

In [ ]:
stopwords = set(STOPWORDS)

wordcloud = WordCloud(
    background_color='white',
    stopwords=stopwords,
    max_words=500,
    max_font_size=200, 
    width=1000, height=800,
    random_state=42,
).generate(" ".join(df_2018['name'].dropna().astype(str)))

print(wordcloud)
fig = plt.figure(figsize = (12,14))
plt.imshow(wordcloud)
plt.title("WORD CLOUD - REGION DESCRIPTION",fontsize=25)
plt.axis('off')

plt.show()

In [ ]:
state_percentage = df_2018.state.value_counts(normalize=True)

trace = go.Pie(labels=state_percentage.index, values=state_percentage, textinfo='value')
layout = dict(title = 'Project States Percentages')

py.iplot(dict(data=[trace], layout=layout))

In [ ]:
df_2018 = df_2018.loc[df_2018['state'].isin(['failed','successful'])]

In [ ]:
df_2018.country.value_counts(normalize=True)

In [ ]:
df_2018.drop(df_2018[df_2018['country']=='N,0"'].index, inplace=True)


In [ ]:
country_percent = df_2018.country.value_counts(normalize=True)
trace = go.Bar(x=country_percent.index, y=country_percent)
layout = dict(title = 'Percentage of Projects per Country',
              xaxis = dict(title = 'Country'),
              yaxis = dict(title = 'Percentage of Projects'))

py.iplot(dict(data=[trace], layout=layout))

In [ ]:
df_2018.main_category.value_counts(normalize=True)

In [ ]:
plt.subplots(figsize=(19,5))
sns.countplot(df_2018['main_category'])
sns.despine(bottom=True, left=True)

In [ ]:
print('Categories in category: ', df_2018['category'].nunique())
df_2018['category'].value_counts()[:20].plot(kind='barh', 
                                        figsize=(14,6), 
                                        title='Top 20 most popular categories')

In [ ]:
main_cats = df_2018["main_category"].value_counts()
main_cats_failed = df_2018[df_2018["state"] == "failed"]["main_category"].value_counts()
main_cats_sucess = df_2018[df_2018["state"] == "successful"]["main_category"].value_counts()

#First plot
trace0 = go.Bar(
    x=main_cats_failed.index,
    y=main_cats_failed.values,
    name="Failed Categories"
)
#Second plot
trace1 = go.Bar(
    x=main_cats_sucess.index,
    y=main_cats_sucess.values,
    name="Success Categories"
)
#Third plot
trace2 = go.Bar(
    x=main_cats.index,
    y=main_cats.values,
    name="Categories Distribuition"
)

#Creating the grid
fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                          subplot_titles=('Failed','Sucessful', "General Category's"))

#setting the figs
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)

fig['layout'].update(showlegend=True,
                     title="Main Category's Distribuition",
                     bargap=0.05)
iplot(fig)

In [ ]:
# Creating subplots for each category
sns.set_palette(sns.cubehelix_palette(8))
fig = plt.figure(figsize = (15,15))
fig.add_subplot(3, 3, 1)
plt.title("Product Design")
sns.countplot(df_2018[df_2018["category"] == "Product Design"]["state"])
fig.add_subplot(3, 3, 2)
plt.title("Tabletop Games")
sns.countplot(df_2018[df_2018["category"] == "Tabletop Games"]["state"])
fig.add_subplot(3, 3, 3)
plt.title("Shorts")
sns.countplot(df_2018[df_2018["category"] == "Shorts"]["state"])
fig.add_subplot(3, 3, 4)
plt.title("Video Games")
sns.countplot(df_2018[df_2018["category"] == "Video Games"]["state"])
fig.add_subplot(3, 3, 5)
plt.title("Food")
sns.countplot(df_2018[df_2018["category"] == "Food"]["state"])
fig.add_subplot(3, 3, 6)
plt.title("Film & Video")
sns.countplot(df_2018[df_2018["category"] == "Film & Video"]["state"])
fig.add_subplot(3, 3, 7)
plt.title("Documentary")
sns.countplot(df_2018[df_2018["category"] == "Documentary"]["state"])
fig.subplots_adjust(wspace = 0.4, hspace= 0.8, right= 0.9, left = 0.125)

for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=50)
    
plt.show()

In [ ]:
failed_film = df_2018[(df_2018['main_category'] == 'Film & Video') & 
                      (df_2018['state'] == 'failed')]
failed_publishing = df_2018[(df_2018['main_category'] == 'Publishing') & 
                      (df_2018['state'] == 'failed')]
failed_music = df_2018[(df_2018['main_category'] == 'Music') & 
                      (df_2018['state'] == 'failed')]

plt.figure(figsize=(14,16))

plt.subplot(3,1,1)
ax0 = sns.countplot(x='category', data=failed_film, color='coral')
ax0.set_xticklabels(ax0.get_xticklabels(),rotation=90)
ax0.set_title("Film & Video Most Fail Category's ", fontsize=22)
ax0.set_xlabel("", fontsize=15)
ax0.set_ylabel("Counts", fontsize=15)
sizes=[]
for p in ax0.patches:
    height = p.get_height()
    sizes.append(height)
    ax0.text(p.get_x()+p.get_width()/2.,
            height + 2,
            '{:1.2f}%'.format(height/len(failed_film)*100),
            ha="center", fontsize=10) 
ax0.set_ylim(0, max(sizes) * 1.15)

plt.subplot(3,1,2)
ax1 = sns.countplot(x='category', data=failed_publishing, color='coral')
ax1.set_xticklabels(ax1.get_xticklabels(),rotation=90)
ax1.set_title("Publishing Most Fail Category's", fontsize=22)
ax1.set_xlabel("", fontsize=17)
ax1.set_ylabel("Counts", fontsize=17)
sizes=[]
for p in ax1.patches:
    height = p.get_height()
    sizes.append(height)
    ax1.text(p.get_x()+p.get_width()/2.,
            height + 2,
            '{:1.2f}%'.format(height/len(failed_publishing)*100),
            ha="center", fontsize=10) 
ax1.set_ylim(0, max(sizes) * 1.15)

plt.subplot(3,1,3)
ax2 = sns.countplot(x='category', data=failed_music, 
                    color='coral')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=90)
ax2.set_title("Music Most Fail Category's", fontsize=22)
ax2.set_xlabel("Category Names", fontsize=17)
ax2.set_ylabel("Counts", fontsize=17)
sizes=[]
for p in ax2.patches:
    height = p.get_height()
    sizes.append(height)
    ax2.text(p.get_x()+p.get_width()/2.,
            height + 2,
            '{:1.2f}%'.format(height/len(failed_music)*100),
            ha="center", fontsize=10) 
ax2.set_ylim(0, max(sizes) * 1.15)

plt.subplots_adjust(wspace = 0.5, hspace = 0.6,top = 0.9)

plt.show()

In [ ]:
pledged_sum = {}
for category in list(set(df_2018['main_category'])):
    amount = df_2018[df_2018['main_category'] == category]['usd_pledged_real'].sum()
    pledged_sum[category] = amount

# Create dataframe
cate_df = pd.Series(pledged_sum)
cate_df = pd.DataFrame(cate_df)
cate_df = cate_df.rename(columns = {0:"pledged_sum"})

cate_df.head()

In [ ]:
cate_df = cate_df.sort_values('pledged_sum',  ascending = False)
plt.subplots(figsize = (20,5))
sns.set_style("whitegrid")
sns.barplot(cate_df['pledged_sum'].index, y= cate_df['pledged_sum'] ,
            palette="Set1",saturation = 0.5)
sns.despine(right = True, top = True)

In [ ]:
cate_count = {}
for category in list(set(df_2018['main_category'])):
    count = df_2018[df_2018['main_category'] == category]['main_category'].count()
    cate_count[category] = count
    
cate_df['count'] = pd.Series(cate_count)

cate_df.head()

In [ ]:
cate_df = cate_df.sort_values('count',  ascending = False)
plt.subplots(figsize = (20,5))
sns.set_style("whitegrid")
sns.barplot(cate_df['count'].index, y= cate_df['count'] ,
            palette="Set1",saturation = 0.5)
sns.despine(right = True, top = True)